In [ ]:
# Load the MacroHFT dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math

In [ ]:
# Load the dataset
df = pd.read_feather("data_1/ETHUSDT/df_train.feather")

# Analyze the dataset
print("Dataset shape:", df.shape)
print("Available columns:", df.columns.values)
print("Sample data:\n", df.head())

# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values[missing_values > 0])

# Handle missing values if any
if df.isnull().sum().sum() > 0:
    df = df.fillna(df.median())

# Visualize the target variable (price_spread)
plt.figure(figsize=(15, 6))
plt.plot(df['price_spread'].values[:200])
plt.title('Price Spread (Target Variable) - First 200 Data Points')
plt.xlabel('Time Steps')
plt.ylabel('Price Spread')
plt.grid(True)
plt.show()

# Show the distribution of the target variable
plt.figure(figsize=(10, 6))
plt.hist(df['price_spread'], bins=50)
plt.title('Distribution of price_spread')
plt.xlabel('price_spread')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Show autocorrelation of the target variable
from pandas.plotting import autocorrelation_plot
plt.figure(figsize=(10, 6))
autocorrelation_plot(df['price_spread'].values[:1000])
plt.title('Autocorrelation of price_spread')
plt.grid(True)
plt.show()


Dataset shape: (525600, 71)
Available columns: ['timestamp' 'ask1_price' 'ask1_size' 'bid1_price' 'bid1_size'
 'ask2_price' 'ask2_size' 'bid2_price' 'bid2_size' 'ask3_price'
 'ask3_size' 'bid3_price' 'bid3_size' 'ask4_price' 'ask4_size'
 'bid4_price' 'bid4_size' 'ask5_price' 'ask5_size' 'bid5_price'
 'bid5_size' 'volume' 'bid1_size_n' 'bid2_size_n' 'bid3_size_n'
 'bid4_size_n' 'bid5_size_n' 'ask1_size_n' 'ask2_size_n' 'ask3_size_n'
 'ask4_size_n' 'ask5_size_n' 'wap_1' 'wap_2' 'wap_balance' 'buy_spread'
 'sell_spread' 'buy_volume' 'sell_volume' 'volume_imbalance'
 'price_spread' 'sell_vwap' 'buy_vwap' 'log_return_bid1_price'
 'log_return_bid2_price' 'log_return_ask1_price' 'log_return_ask2_price'
 'log_return_wap_1' 'log_return_wap_2' 'ask1_price_trend_60'
 'bid1_price_trend_60' 'buy_spread_trend_60' 'sell_spread_trend_60'
 'wap_1_trend_60' 'wap_2_trend_60' 'buy_vwap_trend_60'
 'sell_vwap_trend_60' 'volume_trend_60' 'open' 'high' 'low' 'close' 'kmid'
 'klen' 'kmid2' 'kup' 'kup2' 'klow' 

In [ ]:
# Analyze correlations to select features most relevant to price_spread
correlation_with_target = df.corr()['price_spread'].abs().sort_values(ascending=False)
print("Correlations with price_spread:\n", correlation_with_target.head(15))

# Select top 10 features (excluding the target itself)
selected_features = [
    'log_return_bid1_price',  # Highest correlation with price_spread
    'ask4_size_n',
    'kup2',
    'kup',
    'log_return_ask2_price',
    'klow2',
    'ask1_size_n',
    'ask3_size_n',
    'bid1_size_n',
    'bid3_size_n'
]
target_column = 'price_spread'

print("Selected features:", selected_features)

# Extract features and target
X = df[selected_features].values
y = df[target_column].values.reshape(-1, 1)

# Normalize the data
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

# Show the distribution of normalized target
plt.figure(figsize=(12, 6))
plt.hist(y_scaled, bins=50)
plt.title('Distribution of Target Values (price_spread)')
plt.xlabel('Scaled price_spread')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


Correlations with price_spread:
 price_spread            1.000000
wap_balance             0.447740
klen                    0.421304
kup                     0.310378
sell_spread_trend_60    0.293830
sell_spread             0.287040
buy_spread_trend_60     0.281661
klow                    0.276573
buy_spread              0.260797
ask1_size_n             0.166287
bid1_size_n             0.163629
ask5_size_n             0.162290
bid3_size_n             0.161270
ask4_size_n             0.160270
ask3_size_n             0.159779
Name: price_spread, dtype: float64
Selected features: ['log_return_bid1_price', 'ask4_size_n', 'kup2', 'kup', 'log_return_ask2_price', 'klow2', 'ask1_size_n', 'ask3_size_n', 'bid1_size_n', 'bid3_size_n']


In [ ]:
# Split data into training and testing sets (80% train, 20% test)
train_size = int(len(X_scaled) * 0.8)
X_train = X_scaled[:train_size]
y_train = y_scaled[:train_size]
X_test = X_scaled[train_size:]
y_test = y_scaled[train_size:]

print(f"Training data shape: X={X_train.shape}, y={y_train.shape}")
print(f"Testing data shape: X={X_test.shape}, y={y_test.shape}")

# Create sequences for LSTM
def create_sequences(x, y, time_steps=30):
    """
    Create sequences of data suitable for LSTM training
    Args:
        x: Features array (samples × features)
        y: Target array (samples × 1)
        time_steps: Number of time steps to look back
    
    Returns:
        X_seq: Sequence of features (samples × time_steps × features)
        y_seq: Target values (samples × 1)
    """
    X_seq, y_seq = [], []
    for i in range(len(x) - time_steps):
        X_seq.append(x[i:(i + time_steps)])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

# Create sequences
time_steps = 30  # Look back 30 time steps
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, time_steps)

print(f"Training sequences shape: X={X_train_seq.shape}, y={y_train_seq.shape}")
print(f"Testing sequences shape: X={X_test_seq.shape}, y={y_test_seq.shape}")

# Create PyTorch datasets and loaders
batch_size = 64

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_seq)
y_train_tensor = torch.FloatTensor(y_train_seq)
X_test_tensor = torch.FloatTensor(X_test_seq)
y_test_tensor = torch.FloatTensor(y_test_seq)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Training data shape: X=(420480, 10), y=(420480, 1)
Testing data shape: X=(105120, 10), y=(105120, 1)
Training sequences shape: X=(420450, 30, 10), y=(420450, 1)
Testing sequences shape: X=(105090, 30, 10), y=(105090, 1)


In [ ]:
import torch.nn as nn
import torch.optim as optim

class VanillaLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1):
        super(VanillaLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) 
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Model parameters
input_size = len(selected_features)  # Number of features
hidden_size = 64
output_size = 1

# Create the model
model = VanillaLSTM(input_size, hidden_size, output_size)
print(model)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


VanillaLSTM(
  (lstm): LSTM(10, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [ ]:
# Training function with early stopping
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=100, early_stopping_patience=10):
    # Check if GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Move model to device
    model.to(device)
    
    # Lists to store metrics
    train_losses = []
    test_losses = []
    
    # Early stopping variables
    min_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            # Move data to device
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                # Move data to device
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                
                # Forward pass
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                test_loss += loss.item()
        
        avg_test_loss = test_loss / len(test_loader)
        test_losses.append(avg_test_loss)
        
        # Print progress
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.6f}, '
              f'Test Loss: {avg_test_loss:.6f}')
        
        # Early stopping check
        if avg_test_loss < min_val_loss:
            min_val_loss = avg_test_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= early_stopping_patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break
    
    # Load the best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    # Plot the loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(test_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return model, train_losses, test_losses

# Train the model
trained_model, train_losses, test_losses = train_model(
    model, 
    train_loader, 
    test_loader, 
    criterion, 
    optimizer, 
    num_epochs=100,
    early_stopping_patience=10
)


Using device: cpu
Epoch [1/100], Train Loss: 0.000042, Test Loss: 0.000019
Epoch [2/100], Train Loss: 0.000039, Test Loss: 0.000018
Epoch [3/100], Train Loss: 0.000038, Test Loss: 0.000020
Epoch [4/100], Train Loss: 0.000037, Test Loss: 0.000017
Epoch [5/100], Train Loss: 0.000037, Test Loss: 0.000017
Traceback (most recent call last):
  File "/Users/lehoangsang/.vscode/extensions/ms-python.python-2025.2.0-darwin-arm64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 95, in <module>
  File "<string>", line 30, in train_model
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/

In [ ]:
# Evaluate the model
def evaluate_model(model, X_test_tensor, y_test_tensor, y_scaler):
    # Set model to evaluation mode
    model.eval()
    
    # Check if GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    X_test_tensor = X_test_tensor.to(device)
    
    # Make predictions
    with torch.no_grad():
        y_pred_scaled = model(X_test_tensor).cpu().numpy()
    
    # Inverse transform to get actual values
    y_pred = y_scaler.inverse_transform(y_pred_scaled)
    y_actual = y_scaler.inverse_transform(y_test_tensor.numpy())
    
    # Calculate metrics
    mse = mean_squared_error(y_actual, y_pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_actual, y_pred)
    r2 = r2_score(y_actual, y_pred)
    
    print(f"Mean Squared Error (MSE): {mse:.6f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
    print(f"Mean Absolute Error (MAE): {mae:.6f}")
    print(f"R² Score: {r2:.6f}")
    
    # Plot actual vs predicted values
    plt.figure(figsize=(15, 8))
    plt.plot(y_actual[:200], label='Actual')
    plt.plot(y_pred[:200], label='Predicted')
    plt.xlabel('Time Steps')
    plt.ylabel('Price Spread')
    plt.title('Actual vs Predicted Price Spread - First 200 Predictions')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Plot the distribution of prediction errors
    errors = y_actual - y_pred
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=50)
    plt.xlabel('Prediction Error')
    plt.ylabel('Frequency')
    plt.title('Distribution of Prediction Errors')
    plt.grid(True)
    plt.show()
    
    return y_pred, y_actual, mse, rmse, mae, r2

# Evaluate the trained model
y_pred, y_actual, mse, rmse, mae, r2 = evaluate_model(trained_model, X_test_tensor, y_test_tensor, y_scaler)


Mean Squared Error (MSE): 0.000000
Root Mean Squared Error (RMSE): 0.000010
Mean Absolute Error (MAE): 0.000003
R² Score: -0.012692


In [ ]:
# Function to make predictions on new data
def predict_price_spread(model, new_data, X_scaler, y_scaler, time_steps=30):
    """
    Make price_spread predictions on new data
    
    Args:
        model: Trained LSTM model
        new_data: New feature data (should have the same features as training data)
        X_scaler: The scaler used to normalize the training features
        y_scaler: The scaler used to normalize the target values
        time_steps: Number of time steps used in the model
        
    Returns:
        predictions: Predicted price_spread values
    """
    # Set model to evaluation mode
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Scale the new data
    new_data_scaled = X_scaler.transform(new_data)
    
    # Create sequences for prediction
    sequences = []
    for i in range(len(new_data_scaled) - time_steps + 1):
        sequences.append(new_data_scaled[i:i+time_steps])
    
    # Convert to tensor
    sequences_tensor = torch.FloatTensor(sequences).to(device)
    
    # Make predictions
    with torch.no_grad():
        predicted_scaled = model(sequences_tensor).cpu().numpy()
    
    # Inverse transform the predictions
    predictions = y_scaler.inverse_transform(predicted_scaled)
    
    return predictions.flatten()

# Example of using the prediction function (assuming you have new data)
# This would be applied to your test/validation data
new_data = df[selected_features].values[-100:]  # Last 100 data points as an example
predictions = predict_price_spread(trained_model, new_data, X_scaler, y_scaler)

# # Plot the predictions
plt.figure(figsize=(15, 6))
plt.plot(predictions, label='Predicted price_spread')
plt.xlabel('Time Steps')
plt.ylabel('Price Spread')
plt.title('Predicted Price Spread Values')
plt.legend()
plt.grid(True)
plt.show()


Traceback (most recent call last):
  File "/Users/lehoangsang/.vscode/extensions/ms-python.python-2025.2.0-darwin-arm64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 54, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matplotlib/pyplot.py", line 612, in show
    return _get_backend_mod().show(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matplotlib/backend_bases.py", line 3553, in show
    cls.mainloop()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matplotlib/backends/backend_macosx.py", line 178, in start_main_loop
    with _allow_interrupt_macos():
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/contextlib.py", line 144, 